https://work.datafountain.cn/forum?id=223&type=2&source=1

看了一下，基本上物品的特征就是id，用户倒是有一些标签。

然后呢，这版实现，双塔的层数是一样的。可不可以不一样？

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from torch.nn.init import constant_, normal_, xavier_normal_
from torch.optim import Adam


import sys
import os
import logging

In [2]:
class DataLoder(object):

    def __init__(self, x, y, batch_size=1024) -> None:
        super().__init__()
        self.pr = 0 ## 我估计是present的意思。
        self.batch_size = batch_size
        self.x = x
        self.y = y

    def __iter__(self):
        return self

    def __next__(self):
        if self.pr < len(self.x):
            start = self.pr
            end = self.pr + self.batch_size
            if end > len(self.x):
                end = len(self.x)
            self.pr = end
            return self.x[start:end], self.y[start:end]
        else:
            self.pr = 0
            raise StopIteration

class Logger(object):

    def __init__(self, filename):

        self.logger = logging.getLogger(filename)
        self.logger.setLevel(logging.DEBUG)
        formatter = logging.Formatter('%(asctime)s.%(msecs)03d: %(message)s',
                                      datefmt='%Y-%m-%d %H:%M:%S')

        # write into file
        fh = logging.FileHandler(filename)
        fh.setLevel(logging.DEBUG)
        fh.setFormatter(formatter)

        # show on console
        ch = logging.StreamHandler(sys.stdout)
        ch.setLevel(logging.DEBUG)
        ch.setFormatter(formatter)

        # add to Handler
        self.logger.addHandler(fh)
        self.logger.addHandler(ch)


    def _flush(self):
        for handler in self.logger.handlers:
            handler.flush()

    def info(self, message):
        self.logger.info(message)
        self._flush()


In [3]:
class Config(object):

    def __init__(self) -> None:
        self.lr = 0.001
        self.dropout = 0.3
        self.batch_size = 1024
        self.epochs = 50
        self.embedding_size = 10
        self.mlp_hidden_size = [256, 256, 256]
        self.user_feature_num = 5  # ['user_id', 'age', 'gender', 'occupation', 'zip']
        self.item_feature_num = 1  # ['movie_id']

args = Config()
DEBUG = False
if DEBUG:
    args.epochs = 2

In [4]:
!head ../originalDataset/ml-100k.txt

# 数据预处理

In [5]:
# 定义用户侧，商品侧特征，label域
user_features = ['user_id', 'gender', 'age', 'occupation', 'zip']
item_features = ['movie_id']
label_features = 'label'

# 读取数据
data = pd.read_table("../originalDataset/ml-100k.txt") # pd.read_csv("../originalDataset/ml-100k.txt", sep='\t')
data['label'] = (data['rating'] > 3).astype(int) ## label就是3分以上与否，说白了就是打高分。

# 为类别属性进行id转换, 让其公用一套id
## 所有的类别型特征，为什么要公用同一套id呢？有什么道理吗？
## 或许看到后面，你会理解的。
shift = 0  # 记录当前已经编码到的id 
for feature in user_features + item_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + shift
    shift += data[feature].nunique()

# 打乱数据
## 我在别的地方看到一个数据划分方式，先按时间排序，再八二开。
## 我觉得这有一定道理，因为后来的数据就是OOT了。
## 今天就先不理会这部分了。以后慢慢理会。
data = data.sample(frac=1).reset_index()

# 训练集验证集划分，比例为8：2
split = int(len(data) * 0.8)
train_data = data.loc[:split]
test_data = data.loc[split:]

# 定义训练集和验证集
X_train, y_train = train_data[user_features +
                              item_features], train_data[label_features]
X_test, y_test = test_data[user_features +
                           item_features], test_data[label_features]


# 构造Dataloader，用于对数据进行分batch
train_batchs = DataLoder(X_train, y_train, batch_size=args.batch_size)
test_batchs = DataLoder(X_test, y_test, batch_size=args.batch_size)

# MLP层

In [6]:
# MLP层
class MLPLayers(nn.Module):
    def __init__(self, layers, dropout=0, activation='Tanh', bn=False, init_method=None):
        super(MLPLayers, self).__init__()
        self.layers = layers
        self.dropout = dropout
        self.activation = activation
        self.use_bn = bn
        self.init_method = init_method

        mlp_modules = []
        for _, (input_size, output_size) in enumerate(zip(self.layers[:-1], self.layers[1:])): 
            ## 这些个layers都是input, output size
            mlp_modules.append(nn.Dropout(p=self.dropout))
            mlp_modules.append(nn.Linear(input_size, output_size))
            if self.use_bn:
                mlp_modules.append(nn.BatchNorm1d(num_features=output_size))
            mlp_modules.append(nn.Tanh())
            
        self.mlp_layers = nn.Sequential(*mlp_modules) ## 前面弄完，这个就是真正的网络的层了。
        if self.init_method is not None:
            self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            if self.init_method == 'norm':
                normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)

    def forward(self, input_feature):
        return self.mlp_layers(input_feature)

In [7]:
# DSSM 模型
class DSSM(nn.Module):
    def __init__(self, user_features=None, item_features=None, mlp_hidden_size=[256, 256],
                 embedding_size=10, num_feature=None):
        super().__init__()
        # load parameters info
        self.mlp_hidden_size = mlp_hidden_size ## [256, 256]
        self.dropout_prob = args.dropout

        self.user_feature = user_features
        self.item_feature = item_features

        self.user_feature_num = len(user_features) ## 5
        self.item_feature_num = len(item_features) ## 1

        self.embedding_size = embedding_size ## 10
        self.num_feature = num_feature ## 这个值为多少呢？说白了就是shift变量，就是整套数据里面所有类别型变量的总类别数。

        self.embed = nn.Embedding(num_feature, self.embedding_size) ## ?->10的一个算是映射吧

        user_size_list = [self.embedding_size *
                          self.user_feature_num] + self.mlp_hidden_size ## [10*5, 256, 256]
        item_size_list = [self.embedding_size *
                          self.item_feature_num] + self.mlp_hidden_size ## [10*1, 256, 256]
        
        # define layers and loss
        self.user_mlp_layers = MLPLayers(
            user_size_list, self.dropout_prob, bn=True)
        self.item_mlp_layers = MLPLayers(
            item_size_list, self.dropout_prob, bn=True)

        self.loss = nn.BCELoss()
        self.sigmod = nn.Sigmoid()

        # parameters initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            xavier_normal_(module.weight.data)
        elif isinstance(module, nn.Linear):
            xavier_normal_(module.weight.data)
            if module.bias is not None:
                constant_(module.bias.data, 0)
                
    def embed_user_input(self, x):
        results = []
        user_x = x[self.user_feature]
        for col in user_x.columns:
            idx = torch.LongTensor(user_x[col].tolist())
            results.append(self.embed(idx).unsqueeze(1))
        return torch.cat(results, dim=1)

    def embed_item_input(self, x):
        results = []
        item_x = x[self.item_feature]
        for col in item_x.columns:
            idx = torch.LongTensor(item_x[col].tolist())
            results.append(self.embed(idx).unsqueeze(1))
        return torch.cat(results, dim=1)
    
    def forward(self, x):
        embed_user = self.embed_user_input(x)
        embed_item = self.embed_item_input(x)

        batch_size = embed_item.shape[0]
        user_dnn_out = self.user_mlp_layers(embed_user.view(batch_size, -1))
        item_dnn_out = self.item_mlp_layers(embed_item.view(batch_size, -1))
        score = torch.cosine_similarity(user_dnn_out, item_dnn_out, dim=1)

        sig_score = self.sigmod(score)
        return sig_score.squeeze()

    def calculate_loss(self, x, y):
        label = torch.FloatTensor(y.tolist())
        output = self.forward(x)
        return self.loss(output, label)

In [8]:
torch.Tensor([[1]]).squeeze()

tensor(1.)

In [9]:
# 创建模型
model = DSSM(user_features=user_features, item_features=item_features, mlp_hidden_size=args.mlp_hidden_size,
             embedding_size=args.embedding_size, num_feature=shift)

# 创建优化器
optimizer = Adam(params=model.parameters(), lr=args.lr)

# 创建日志
logger = Logger('log.txt') 

# 训练

for epoch in range(args.epochs):
    total_loss = None
    model.train()
    for idx, (x, y) in enumerate(train_batchs):
        optimizer.zero_grad()
        losses = model.calculate_loss(x, y)
        losses.backward()
        optimizer.step()
        total_loss = losses.item() if total_loss is None else total_loss + losses.item()
    model.eval()
    preds = []
    trues = []
    with torch.no_grad(): ## torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度。
        for idx, (x, y) in enumerate(test_batchs):
            pred = model(x)
            preds.extend(pred.numpy())
            trues.extend(y.tolist())

    auc = roc_auc_score(trues, preds)
    loss = log_loss(trues, preds)
    logger.info("Epoch {:d} Loss {:4f} AUC {:4f} LogLoss {:4f}".format(epoch, total_loss, auc, loss))

2022-05-14 14:38:33.674: Epoch 0 Loss 54.800395 AUC 0.518149 LogLoss 0.693298
2022-05-14 14:38:38.363: Epoch 1 Loss 54.567612 AUC 0.556224 LogLoss 0.687731
2022-05-14 14:38:43.027: Epoch 2 Loss 53.433128 AUC 0.623684 LogLoss 0.667887
2022-05-14 14:38:47.819: Epoch 3 Loss 51.341466 AUC 0.680786 LogLoss 0.643096
2022-05-14 14:38:52.577: Epoch 4 Loss 49.465626 AUC 0.712640 LogLoss 0.625642
2022-05-14 14:38:57.385: Epoch 5 Loss 47.994871 AUC 0.738300 LogLoss 0.609770
2022-05-14 14:39:02.089: Epoch 6 Loss 46.918527 AUC 0.753753 LogLoss 0.599737
2022-05-14 14:39:06.794: Epoch 7 Loss 46.268345 AUC 0.764355 LogLoss 0.592893
2022-05-14 14:39:11.511: Epoch 8 Loss 45.905574 AUC 0.770129 LogLoss 0.589347
2022-05-14 14:39:16.265: Epoch 9 Loss 45.719941 AUC 0.772257 LogLoss 0.587734
2022-05-14 14:39:20.978: Epoch 10 Loss 45.567423 AUC 0.773771 LogLoss 0.586722
2022-05-14 14:39:25.701: Epoch 11 Loss 45.408663 AUC 0.774698 LogLoss 0.586207
2022-05-14 14:39:30.406: Epoch 12 Loss 45.358069 AUC 0.774842 